In [1]:
import pandas as pd
import glob
import os

c:\Users\aguer\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
carpeta = "produccion-energetica"       # Carpeta donde están tus excels de instalaciones
archivo_hdd = "HDD_por_dia_promedio.xlsx"

In [ ]:
# --- 1. Leer archivo HDD ---
hdd = pd.read_excel(archivo_hdd, engine="openpyxl")
hdd["Date"] = hdd["Date"].astype(str).str.zfill(5)

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [ ]:
! conda install -c anaconda openpyxl=3.1.5


In [ ]:
resultados = []

# --- 3. Recorrer todos los Excel de la carpeta ---
for ruta in glob.glob(os.path.join(carpeta, "*.xlsx")):
    nombre_instalacion = os.path.splitext(os.path.basename(ruta))[0]
    print(f"Procesando instalación: {nombre_instalacion}")

    df = pd.read_excel(ruta)
    df.columns = df.columns.str.strip()  # Limpia espacios en nombres

    # Normalizar columnas
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    df["EnergiaCaldera(MW-hr)"] = df["EnergiaCaldera(MW-hr)"].astype(str).str.replace(",", ".").astype(float)

    # Crear columna Date con formato día-mes
    df["Date"] = df["Timestamp"].dt.strftime("%d-%m")

    # Agrupar por día y sacar media (por si hay varios años)
    df_media = df.groupby("Date", as_index=False)["EnergiaCaldera(MW-hr)"].mean()

    # Añadir nombre de instalación
    df_media["Instalacion"] = nombre_instalacion

    # Unir con HDD
    combinado = pd.merge(df_media, hdd, on="Date", how="left")

    # Reordenar columnas
    combinado = combinado[["Date", "Instalacion", "HDD", "EnergiaCaldera(MW-hr)"]]

    resultados.append(combinado)

In [ ]:
# --- 4. Unir todo ---
df_final = pd.concat(resultados, ignore_index=True)

In [ ]:
print(df_final.head(10))